In [32]:
import spotipy
import spotipy.util as util


client_id = 'b69a9985fa8842deb0691b2d0e3f0b69'
client_secret = 'd9e9ae2924174c139a5a9ccb303f9f3a'
redirect_uri = 'http://localhost/'

username = '22mrmbu7oumkrb56tcsclawdi'

token = util.prompt_for_user_token(username, 'user-library-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)
tracks = []
offset = 0
while True:
    track_set = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
    if track_set:
        tracks += track_set
    else:
        break
    offset += 50
ids = []

for i in range(len(tracks)):
    tracks[i] = tracks[i]['track']
    ids.append(tracks[i]['id'])




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b69a9985fa8842deb0691b2d0e3f0b69&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=user-library-read in your browser




Enter the URL you were redirected to:  http://localhost/?code=AQDNYa5pL6qSpBZjdnRMly-ycXq1_33yT25cpNi673w-2KhMOUs1FWozOcHkz67tqPYTecrt4S1weGTKPJDgYj_FGyJK49NuZYtXriNnU3Kk-VllHzc6muZZADLCHbtQjffFKB4ERkyJOnRcE9tBZuJPIB_GJAsS59A7sZnvIq7VJFLIv8L9zxzOKXivXJdsJPDTwD5TTtpa1FOF


In [33]:
import pandas as pd

data = {
    'acousticness': [],
    'danceability': [],
#     'duration_ms': [],
    'energy': [],
    'instrumentalness': [],
#     'key': [],
#     'liveness': [],
    'loudness': [],
#     'mode': [],
#     'speechiness': [],
    'tempo': [],
#     'time_signature': [],
    'valence': [],
}

def chunks(n, x):
    return [x[i:i + n] for i in range(0, len(x), n)]

for chunk in chunks(100, ids):
    track_set = sp.audio_features(tracks=chunk)
    for track in track_set:
        for key in data:
            data[key].append(track[key])

artist_genres = dict()
track_artists = []
for track in tracks:
    artists = [x['id'] for x in track['artists']]
    track_artists.append(artists)
    for artist in artists:
        if artist not in artist_genres:
            artist_genres[artist] = None

for chunk in chunks(50, list(artist_genres.keys())):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']

data['genres'] = []
for artists in track_artists:
    genres = set()
    for id in artists:
        genres.update(artist_genres[id])
    data['genres'].append(genres)
    
df = pd.DataFrame(data=data, index=ids)

In [72]:
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

features = df.loc[:, df.columns != 'genres']
# display(features)
# g = sns.PairGrid(features)
# g.map_diag(sns.kdeplot)
# g.map_offdiag(sns.kdeplot, n_levels=10);

X = preprocessing.scale(features.values)
# for x in X.transpose():
#     sns.distplot(x)
#     plt.show()

import sklearn.cluster

n_clusters = 6
df['cluster'] = sklearn.cluster.AgglomerativeClustering(n_clusters=n_clusters).fit_predict(X)

In [73]:
counts = {'cluster': [], 'genre': [], 'count': []}
for cluster in df['cluster'].unique():
    df_cluster = df[ df['cluster']==cluster ]
    genre_count = dict()
    for genres in df_cluster['genres']:
        for genre in genres:
            if genre in genre_count:
                genre_count[genre] += 1
            else:
                genre_count[genre] = 1
    for genre, count in genre_count.items():
        counts['cluster'].append(cluster)
        counts['genre'].append(genre)
        counts['count'].append(count)

counts = pd.DataFrame(counts)
counts.sort_values('count', ascending=False, inplace=True)
original_counts = counts.copy()

clusters_to_name = set(counts['cluster'].unique())
playlists = dict()
while clusters_to_name:
    cluster = counts.iloc[0]['cluster']
    name = counts.iloc[0]['genre']
    playlists[name] = cluster
    counts = counts[ (counts['cluster']!=cluster) & (counts['genre']!=name) ]
    clusters_to_name.remove(cluster)
    
playlist_tracks = dict()
from collections import OrderedDict
for name in playlists:
    playlist_tracks[name] = df[ df['cluster']==playlists[name] ].index.values
    playlist_genres = original_counts[ original_counts['cluster']==playlists[name] ].sort_values('count', ascending=False)['genre'].values[:10]
    playlist_counts = original_counts[ original_counts['cluster']==playlists[name] ].sort_values('count', ascending=False)['count'].values[:10]
    playlist_counts = list(map(int, playlist_counts))
    playlists[name] = OrderedDict(zip(playlist_genres, playlist_counts))

In [70]:
token = util.prompt_for_user_token(username, 'playlist-modify-private', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

for name in playlist_tracks:
    id = sp.user_playlist_create(username, 'sortify '+name, public=False)['id']
    for chunk in chunks(100, playlist_tracks[name]):
        sp.user_playlist_add_tracks(username, id, chunk)

In [ ]:
classified = [[] for i in range(clusters)]
ids = df.index.values
for i in range(len(df)):
    cluster = model.predict(X[0].reshape(1, -1))[0]
    classified[cluster].append(ids[i])
for i in classified:
    print(len(i))

In [75]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(playlists)

{   'edm': OrderedDict([   ('edm', 433),
                           ('electro house', 412),
                           ('brostep', 260),
                           ('electronic trap', 234),
                           ('big room', 194),
                           ('progressive house', 171),
                           ('tropical house', 166),
                           ('progressive electro house', 158),
                           ('pop', 154),
                           ('bass trap', 148)]),
    'electro house': OrderedDict([   ('electro house', 218),
                                     ('edm', 185),
                                     ('electronic trap', 120),
                                     ('tropical house', 114),
                                     ('brostep', 113),
                                     ('house', 111),
                                     ('pop', 84),
                                     ('big room', 84),
                                     ('progressive hou